# Pull_out_business_by_ID

In [1]:
%load_ext sql
%load_ext lab_black

In [2]:
import sqlite3
import pandas as pd
from tqdm import tqdm_notebook as tqdm

from src.features import paths

In [3]:
path_to_electricity_allocations = paths.get_path_to_data(
    "SME and Residential allocations.xlsx"
)

print(path_to_electricity_allocations)

C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\raw\SM_electricity\SME and Residential allocations.xlsx


In [4]:
path_to_SME_electricity_survey = paths.get_path_to_data(
    "Smart meters SME pre-trial survey data.csv"
)

print(path_to_SME_electricity_survey)

C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\raw\SM_electricity\Smart meters SME pre-trial survey data.csv


In [5]:
path_to_data_database = paths.get_path_to_data("data.db")

print(path_to_data_database)

C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\processed\data.db


# Pull out SME Survey business types

In [6]:
SME_electricity_survey = pd.read_csv(
    path_to_SME_electricity_survey, usecols=[0, 1], index_col=[0]
)

SME_electricity_survey.rename(
    columns={
        "Question 611: INT:Code organistation s business as appropriate to the following": "SME_type"
    },
    inplace=True,
)

SME_electricity_survey

,SME_type
ID,
1023,5
1026,5
1050,4
1056,2
1101,5
...,...
7348,6
7357,4
7369,4


In [7]:
SME_electricity_survey_grouped = SME_electricity_survey.groupby("SME_type")

In [8]:
SME_electricity_survey_grouped.get_group(1)

,SME_type
ID,
1637,1
2639,1
4285,1
6458,1
6939,1


# Connect to Database

In [9]:
%sql sqlite:///$path_to_data_database

'Connected: @C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\processed\\data.db'

In [10]:
conn = sqlite3.connect(path_to_data_database)

conn

In [11]:
cursor = conn.cursor()

In [12]:
%%sql

SELECT * FROM sqlite_master

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\processed\data.db
Done.


type,name,tbl_name,rootpage,sql
table,electricity_data_ordered,electricity_data_ordered,2,"CREATE TABLE electricity_data_ordered ( ID TEXT, Date TEXT, Time TEXT, Consumption FLOAT)"
table,electricity_average_consumption,electricity_average_consumption,1629893,"CREATE TABLE electricity_average_consumption ( Date TEXT, Time TEXT, Consumption FLOAT)"


# Pull out by ID

In [13]:
%%time

for business_type in range(1, 7):

    select = SME_electricity_survey_grouped.get_group(business_type)
    
    select_IDs = tuple(select.index)
    print(f"Calculating an average demand profile for group {business_type}...")
    sql = f"""SELECT Date, Time, AVG(Consumption) FROM electricity_data_ordered 
            WHERE ID in {select_IDs}
            GROUP BY Date, Time
            """
    group = pd.read_sql_query(sql, conn)
    print("Reading of SQL query complete\n")
    
    print("Calculating hourly demand...")
    group["Date"] = pd.to_datetime(group["Date"])
    group["Time"] = pd.to_datetime(group["Time"], format="%H:%M:%S").dt.time
    group.set_index(["Date", "Time"], inplace=True)

    hourly_demand = group["AVG(Consumption)"].iloc[0::2].values + group["AVG(Consumption)"].iloc[1::2].values

    # Calculate hourly profiles
    group_hourly = pd.Series(
        hourly_demand,
        index=group.iloc[0::2].index,
    )

    print(f"Storing Data for group {business_type} in csv...\n")
    group_hourly.to_csv(r"C:\Users\RowanM\Downloads\group{}.csv".format(business_type))

Calculating an average demand profile for group 1...
Reading of SQL query complete

Calculating hourly demand...
Storing Data for group 1 in csv...

Calculating an average demand profile for group 2...
Reading of SQL query complete

Calculating hourly demand...
Storing Data for group 2 in csv...

Calculating an average demand profile for group 3...
Reading of SQL query complete

Calculating hourly demand...
Storing Data for group 3 in csv...

Calculating an average demand profile for group 4...
Reading of SQL query complete

Calculating hourly demand...
Storing Data for group 4 in csv...

Calculating an average demand profile for group 5...
Reading of SQL query complete

Calculating hourly demand...
Storing Data for group 5 in csv...

Calculating an average demand profile for group 6...
Reading of SQL query complete

Calculating hourly demand...
Storing Data for group 6 in csv...

Wall time: 6min 36s
